In [2]:
import numpy as np

In [ ]:
class ANN:
    w_l_ij= None
    a_l_ij = None
    input_featureCount = None
    input_count = None
    layer = None
    def __init__(self,inputDim,inputCount):
        self.input_featureCount = inputDim
        self.input_count = inputCount
        Layer = np.array([inputDim])
        return
    def __str__(self):
        return
    def addLayer(self,neuronCount,activationFunction):
        if(self.Layer.shape == (1,)):
            Layer = np.append(Layer,[neuronCount],axis = 0)
            w = np.random.rand(neuronCount,self.input_featureCount)
            w

Neural Network Characteristics
    No of Layers
        No of Neurons
        Activation Function
        No of input
    Loss Function

Layer1    n1 neuron  -> weights depend on input features
Layer2    n2 neuron  -> n1 weights


In [14]:
x = np.array([1])
x = np.append(x,[2],axis = 0)

In [12]:
x

array([1, 2])